<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Prompts

Prompts are the fundemental building block to getting the proper responses from AI models. This module demonstrates how to use common prompt engineering techniques while using Semantic Kernel. If you've ever used ChatGPT or Microsoft Copilot, you're already familiar with prompting. Given a prompt instruction, an LLM will attempt to predict the most likely response. With Semantic Kernel, you can craft applications, services & APIs that execute prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [7]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// Retrieve the configuration settings for the Azure OpenAI service
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];

// Optional OpenAI
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [8]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.11.1"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Create a new Semantic Kernel instance with the Azure OpenAI Chat Completion service
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.11.1

In [9]:
// Create a new Semantic Kernel instance with the Azure OpenAI Chat Completion service
var semanticKernelOpenAI = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion(
        modelId: openAIModelId,
        apiKey: openAIAPIKey)
    .Build();

### Step 2 - Execute a Semantic Kernel Prompt

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper decision frames
* Gather the information & intelligence in order to make a decision
* Recommend decision frameworks to make a high-quality decision
* Provide feedback to evaluate a decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks. 

In [10]:
var simplePrompt = "Provide some decision frameworks that can help improve the quality of decisions.";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simplePrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

1. SWOT Analysis: This involves evaluating the Strengths, Weaknesses, Opportunities, and Threats related to a decision to understand the potential outcomes and risks.

2. Decision Matrix: This framework involves creating a grid to compare various options based on defined criteria, scoring each option, and selecting the one with the highest score.

3. Cost-Benefit Analysis: This involves listing the costs and benefits of each option and comparing them to determine which option provides the greatest net benefit.

4. Pareto Analysis: This technique involves identifying the options that will have the most significant impact by applying the 80/20 rule, which states that 80% of effects come from 20% of the causes.

5. Scenario Planning: This involves creating different plausible future scenarios and exploring how decisions might play out in each scenario to make informed choices.

6. Decision Tree: This involves mapping out different decision paths and the potential outcomes of each path, he

### Step 3 - Execute a Semantic Kernel Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [11]:
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simplePrompt))
{
   Console.Write(streamChunk);
}

1. SWOT Analysis: This framework assesses the Strengths, Weaknesses, Opportunities, and Threats related to a decision. It helps identify internal and external factors that can influence outcomes.

2. Decision Matrix: Utilizes a grid format to evaluate and prioritize multiple options based on specific criteria, assigning weights to each criterion to determine the most favorable option.

3. Cost-Benefit Analysis: Measures the costs versus benefits of different decision options to determine which provides the greatest net value.

4. The Six Thinking Hats: This framework by Edward de Bono involves considering a decision from multiple perspectives, represented as six colored hats, to foster comprehensive and balanced consideration.

5. The Delphi Technique: A method to reach consensus among a group of experts through multiple rounds of anonymous input and feedback, refining decisions based on collective intelligence.

6. Pareto Analysis (80/20 Rule): Focuses on identifying the critical fact

### Step 4 - Execute a Semantic Kernel Prompt with a Custom Prompt Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Tempature or other settings.

In [12]:
// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 750, 
    Temperature = 0.0, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simplePrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

1. SWOT Analysis: This framework helps identify the strengths, weaknesses, opportunities, and threats related to a decision. It provides a comprehensive view of the internal and external factors that can impact the decision.

2. Decision Matrix: This framework helps evaluate and prioritize different options based on specific criteria. Each option is scored based on how well it meets each criterion, and the option with the highest score is selected.

3. Cost-Benefit Analysis: This framework helps assess the financial implications of a decision by comparing the costs and benefits of each option. It helps identify the option that provides the greatest net benefit.

4. Pareto Analysis: This framework helps identify the most significant factors that contribute to a problem or decision. It is based on the Pareto principle, which states that 80% of the effects come from 20% of the causes.

5. Decision Tree: This framework helps visualize the different paths and outcomes of a decision. It help

## This can be run on OpenAI, not just Azure OpenAI 

In [13]:
// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 750, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
await foreach (var streamChunk in semanticKernelOpenAI.InvokePromptStreamingAsync(simplePrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

Certainly! Decision-making frameworks can significantly enhance the quality of decisions by providing structured approaches to evaluate options and outcomes. Here are some widely recognized frameworks:

### 1. **SWOT Analysis**
- **Strengths:** Identify internal strengths.
- **Weaknesses:** Identify internal weaknesses.
- **Opportunities:** Identify external opportunities.
- **Threats:** Identify external threats.

### 2. **PEST Analysis**
- **Political:** Consider political factors and their impact.
- **Economic:** Evaluate economic conditions and trends.
- **Social:** Assess social factors and cultural trends.
- **Technological:** Examine technological advancements and innovations.

### 3. **Decision Matrix (Weighted Scoring)**
- List all options and criteria.
- Assign weights to each criterion based on importance.
- Score each option against each criterion.
- Multiply scores by weights and sum them to get a total score for each option.

### 4. **Cost-Benefit Analysis**
- Identify al

### Step 5 - Execute a Semantic Kernel Prompt with a system prompt (persona)

All the best practices of prompt engineering apply when using Semantic Kernel.  
* Make the prompt more specific
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Tell the AI what to do to avoid doing something wrong
* Provide context to the AI
* Using message roles in chat completion prompts
* Give your AI words of encouragement  

One best practice is to provide a common behaviour across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. These prompts can be dynamically created with a programming language like C#.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. The decision framework responses are much more robust with decision intelligence information.

In [14]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt
var systemPromptForDecisions = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var simplePrompt = "Provide some decision frameworks that can help improve the quality of decisions.";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {simplePrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 750, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

Console.WriteLine("Prompt To Send to the Azure OpenAI Chat Completion Service...");
Console.WriteLine(simpleDecisionPromptTemplate);
Console.WriteLine(string.Empty);
Console.WriteLine("----------------------------------------");
Console.WriteLine(string.Empty);
Console.WriteLine("Response from the Azure OpenAI Chat Completion Service...");


await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simplePrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

Prompt To Send to the Azure OpenAI Chat Completion Service...
System Prompt: You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

Request from the user: Provide some decision frameworks that can help improve the quality of decisions.

----------------------------------------

Response from the Azure OpenAI Chat Completion Service...
Improving the quality of decisions involves using structured frameworks that guide the decision-making process, ensuring that all relevant factors are considered and that the outcomes align with the overall goals. Here are several decision frameworks that can help in various contexts:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**:
   - **Purpose**: To evaluate the internal and external factors affecting a decision.
   - **Process**:
     1. List all Str